In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv
load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')



In [6]:
llm = ChatOpenAI(model = "gpt-5", max_tokens = 1000, temperature = 0)

In [8]:
store: dict[str, ChatMessageHistory] = {}

def get_chat_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [10]:
chain = prompt | llm

In [13]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [14]:
session_id = 'inchan'
response1 = chain_with_history.invoke(
    {"input": "hello how are you?"}, 
    config = {"configurable": 
              {"session_id": session_id}
             }
)

print("AI : ", response1.content)

AI :  Hi! I’m doing well, thanks for asking. How are you? What can I help you with today?


In [15]:
response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Your previous message was: "hello how are you?"


In [16]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: hello how are you?
ai: Hi! I’m doing well, thanks for asking. How are you? What can I help you with today?
human: What was my previous message?
ai: Your previous message was: "hello how are you?"
